# 07wk-2: Pandas – pivot_table, groupby

최규빈  
2023-10-16

In [1]:
import pandas as pd 
import numpy as np

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook/master/data/flights.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58492 entries, 0 to 58491
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MONTH      58492 non-null  int64  
 1   DAY        58492 non-null  int64  
 2   WEEKDAY    58492 non-null  int64  
 3   AIRLINE    58492 non-null  object 
 4   ORG_AIR    58492 non-null  object 
 5   DEST_AIR   58492 non-null  object 
 6   SCHED_DEP  58492 non-null  int64  
 7   DEP_DELAY  57659 non-null  float64
 8   AIR_TIME   57474 non-null  float64
 9   DIST       58492 non-null  int64  
 10  SCHED_ARR  58492 non-null  int64  
 11  ARR_DELAY  57474 non-null  float64
 12  DIVERTED   58492 non-null  int64  
 13  CANCELLED  58492 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 6.2+ MB

`-` 각 변수들에 대한 설명은 아래와 같다. (ChatGPT의 도움을 받아 정리함)

1.  `MONTH`: 비행이 이루어진 월을 나타냄. 1에서 12 사이의 값을 갖음.
2.  `DAY`: 비행이 이루어진 일자를 나타냄. 월에 따라 1~28/29/30/31 사이의
    값을 가질 수 있음.
3.  `WEEKDAY`: 비행이 이루어진 요일을 나타냄. 일반적으로 1(일요일)부터
    7(토요일)까지의 값을 갖음.
4.  `AIRLINE`: 해당 항공편을 운영하는 항공사의 약어나 코드를 나타냄.
5.  `ORG_AIR`: 비행기가 출발하는 공항의 약어나 코드를 나타냄.
6.  `DEST_AIR`: 비행기가 도착하는 공항의 약어나 코드를 나타냄.
7.  `SCHED_DEP`: 원래의 예정된 출발 시간을 나타냄. 시간은 일반적으로
    HHMM 형식으로 표시될 수 있음.
8.  `DEP_DELAY`: 출발 지연 시간을 나타냄. 음수 값은 조기 출발, 양수 값은
    지연을 의미함.
9.  `AIR_TIME`: 실제 공중에서 비행한 시간을 분 단위로 나타냄.
10. `DIST`: 비행 거리를 나타냄. 일반적으로 마일 또는 킬로미터로 표시됨.
11. `SCHED_ARR`: 원래의 예정된 도착 시간을 나타냄. `SCHED_DEP`와 같은
    형식으로 표시될 수 있음.
12. `ARR_DELAY`: 도착 지연 시간을 나타냄. 음수는 조기 도착, 양수는
    지연을 의미함.
13. `DIVERTED`: 항공편이 다른 곳으로 우회되었는지를 나타냄. 1은 우회,
    0은 정상 경로를 의미함.
14. `CANCELLED`: 항공편이 취소되었는지 여부를 나타냄. 1은 취소, 0은
    취소되지 않음을 의미함.

# 5. Groupby

## A. 기본포맷

`# 예제1`: 아래의 예제에서 그룹별 평균을 구하여라.

In [12]:
_category = ['A']*5+['B']*5
_value = np.concatenate([np.random.randn(5), np.random.randn(5)+10])
df = pd.DataFrame({'category':_category, 'value':_value})
df

`-` 방법1: pivot_table 이용

In [14]:
df.pivot_table(index='category',values='value')

In [19]:
df.pivot_table(columns='category',values='value',aggfunc=['mean','sum'])

`-` 방법2 groupby().aggregate()

In [18]:
df.groupby('category').aggregate({'values':np.mean})

***groupby(?)에서 올 수 있는 구조***

-   열의이름
-   \[열의이름,열의이름\]

***aggregate(?)에서 올 수 있는 구조***

-   함수: 함수자체[1]가 오거나, 함수를 의미하는 문자열[2] 이 올 수 있음.
-   리스트: \[함수, 함수\] \# 여기에서 함수자리에는 함수자체, 혹은
    함수문자열 아무것이나 올 수 있음.
-   딕셔너리1: {열의이름:함수}
-   딕셔너리2: {열의이름:\[함수,함수\]}

[1] np.mean,sum

[2] ‘size’, ‘count’, ‘sum’, ‘mean’, ‘median’, ‘min’, ‘max’, ‘std’, ‘var’

In [ ]:
df.groupby(['category']).aggregate({'value':[np.mean,'size']})

In [21]:
df=pd.read_csv('https://raw.githubusercontent.com/PacktPublishing/Pandas-Cookbook/master/data/flights.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58492 entries, 0 to 58491
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   MONTH      58492 non-null  int64  
 1   DAY        58492 non-null  int64  
 2   WEEKDAY    58492 non-null  int64  
 3   AIRLINE    58492 non-null  object 
 4   ORG_AIR    58492 non-null  object 
 5   DEST_AIR   58492 non-null  object 
 6   SCHED_DEP  58492 non-null  int64  
 7   DEP_DELAY  57659 non-null  float64
 8   AIR_TIME   57474 non-null  float64
 9   DIST       58492 non-null  int64  
 10  SCHED_ARR  58492 non-null  int64  
 11  ARR_DELAY  57474 non-null  float64
 12  DIVERTED   58492 non-null  int64  
 13  CANCELLED  58492 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 6.2+ MB

### `#` EX1: \[AIRLINE\] $\to$ {ARR_DELAY:mean}

`-` 방법1: grouby() $\to$ .agg({colname: function})

`# 예제1`: 항공사(AIRLINE)별로 도착지연시간의(ARR_DELAY)의 평균을
구하라.

`-` 풀이1

In [28]:
df.pivot_table(index='AIRLINE',values='ARR_DELAY')

`-` 풀이2

In [30]:
df.groupby(by="AIRLINE").agg({'ARR_DELAY':np.mean})

/tmp/ipykernel_3397090/755890770.py:1: FutureWarning: The provided callable <function mean at 0x7fd81c140280> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df.groupby(by="AIRLINE").agg({'ARR_DELAY':np.mean})

`#`

`# 예제2`: 항공사(AIRLINE)별로 비행취소건수(CANCELLED)의 합계를 구하라.
취소건수가 높은 항공사순으로 정렬하라.

`-` 풀이1

In [36]:
df.pivot_table(index='AIRLINE',values='CANCELLED',aggfunc='sum').sort_values('CANCELLED',ascending=False)

`-` 풀이2

In [37]:
df.groupby('AIRLINE').aggregate({'CANCELLED':'sum'}).sort_values('CANCELLED',ascending=False)

### `#` EX2: \[AIRLINE,WEEKDAY\] $\to$ {CANCELLED:sum}

`-` 방법1

(예시1)

In [ ]:
df.groupby(by=["AIRLINE","WEEKDAY"]).agg({"CANCELLED":np.sum})

(예시2)

In [ ]:
df.groupby(by=["AIRLINE","WEEKDAY"]).agg({"CANCELLED":"sum"})

`-` 방법2

(예시1)

In [ ]:
df.groupby(by=["AIRLINE","WEEKDAY"])[["CANCELLED"]].agg(np.sum)

(예시2)

In [ ]:
df.groupby(by=["AIRLINE","WEEKDAY"])["CANCELLED"].agg("sum")

(예시3)

In [ ]:
df.groupby(by=["AIRLINE","WEEKDAY"])["CANCELLED"].sum()

In [ ]:
df.DIVERTED

### `#` EX3: \[AIRLINE,WEEKDAY\] $\to$ {CANCELLED:sum,mean}, {DIVERTED: sum,mean}

`-` 방법1

(예시1)

In [ ]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({"CANCELLED":[np.sum,np.mean],"DIVERTED":[np.sum,np.mean]})

(예시2)

In [ ]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({"CANCELLED":["sum","mean"],"DIVERTED":["sum","mean"]})

`-` 방법2

(예시1)

In [ ]:
df.groupby(["AIRLINE","WEEKDAY"])[["CANCELLED","DIVERTED"]]\
.agg([np.sum,np.mean])

(예시2)

In [ ]:
df.groupby(["AIRLINE","WEEKDAY"])[["CANCELLED","DIVERTED"]]\
.agg(["sum","mean"])

(예시3) – 사용불가능

### `#` EX4: \[AIRLINE,WEEKDAY\] $\to$ {CANCELLED:sum,mean,count}, {AIR_TIME: mean,var}

`-` 방법1

(예시1)

In [ ]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({'CANCELLED':[np.sum,np.mean,len],'AIR_TIME':[np.mean,np.var]})

(예시2)

In [ ]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({'CANCELLED':["sum","mean","count"],'AIR_TIME':["mean","var"]})

(사용자정의함수)

In [ ]:
df.groupby(["AIRLINE","WEEKDAY"])\
.agg({'CANCELLED':[np.sum,np.mean,len],
      'AIR_TIME':[np.mean,lambda x: np.std(x,ddof=1)**2]})

## 연속형변수를 기준으로 groupby -\> agg

In [ ]:
df.T

In [ ]:
df.DIST.describe()

In [ ]:
df.assign(DIST2 = pd.cut(df.DIST,[-np.inf,391,690,1199,np.inf]))\
.groupby(["AIRLINE","DIST2"]).agg({'CANCELLED':["sum","mean","count"]})

In [ ]:
pd.cut(df.DIST,[-np.inf,400,700,1200,np.inf],labels=['~400','400~700','700~1200','1200~'])

In [ ]:
df.assign(DIST2 = pd.cut(df.DIST,[-np.inf,400,700,1200,np.inf],labels=['~400','400~700','700~1200','1200~']))\
.groupby(["AIRLINE","DIST2"]).agg({'CANCELLED':["sum","mean","count"]})